importing libraries

In [53]:
import sys, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

importing stock prices for the time period December 2015 to November 2020

In [54]:
#apple_full = pd.read_csv('AAPL.csv')
amd_full = pd.read_csv('AMD.csv')
#intel_full = pd.read_csv('INTC.csv')
#microsoft_full = pd.read_csv('MSFT.csv')
#nvidia_full = pd.read_csv('NVDA.csv')

In [62]:
# data we want to predict
subject_data = amd_full

# number of days we will be predicting
#TIP: 20 = a financial month
guess_period = 60

# number of days we will be training the RNN on
#TIP: 20 = a financial month
#TIP2: the maximum logical value is '(subject_data.shape[0]-guess_period)'
train_period = (subject_data.shape[0]-guess_period)

# gives use the values we will be training on
training_set = subject_data.iloc[
    (subject_data.shape[0]-guess_period)-train_period:
    (subject_data.shape[0]-guess_period),
     1:2].values

real_stock_price= subject_data.iloc[(dataset_train.shape[0]-guess_period):, 1:2].values

In [56]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [57]:
n_timesteps = 10

X_train = []
y_train = []

for i in range(n_timesteps, training_set_scaled.shape[0]):
    X_train.append(training_set_scaled[i-n_timesteps:i, 0])
    y_train.append(training_set_scaled[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

In [58]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

building the RNN

In [59]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout 

In [60]:
regressor = Sequential()

n_neurons = 50

regressor.add(LSTM( units = n_neurons, return_sequences= True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(.2))

regressor.add(LSTM( units = n_neurons, return_sequences= True))
regressor.add(Dropout(.2))

regressor.add(LSTM( units = n_neurons, return_sequences= True))
regressor.add(Dropout(.2))

regressor.add(LSTM( units = n_neurons, return_sequences= True))
regressor.add(Dropout(.2))

regressor.add(LSTM( units = n_neurons, return_sequences= False))
regressor.add(Dropout(.2))

regressor.add(Dense(units=1))

regressor.compile(optimizer= 'adam', loss= 'mean_squared_error')

In [61]:
regressor.fit(X_train, y_train, epochs= 5, batch_size= 32)

Epoch 1/5
38/38 [==============================] - 1s 22ms/step - loss: 0.0203
Epoch 2/5
38/38 [==============================] - 1s 20ms/step - loss: 0.0025
Epoch 3/5
38/38 [==============================] - 1s 20ms/step - loss: 0.0022
Epoch 4/5
38/38 [==============================] - 1s 19ms/step - loss: 0.0019
Epoch 5/5
38/38 [==============================] - 1s 20ms/step - loss: 0.0019


making preditions

In [65]:
subject_data_total = subject_data['Open']

lower_bound = len(subject_data_total) - len(test_set) - n_timesteps

inputs = subject_data_total[lower_bound:].values

inputs = inputs.reshape(-1,1)

In [66]:
inputs = sc.transform(inputs)

X_test = []
for i in range(n_timesteps, inputs.shape[0]):
    X_test.append(inputs[i-n_timesteps:i, 0])
X_test = np.array(X_test)

X_test = X_test.reshape( X_test.shape[0], X_test.shape[1], 1)

predicted_stock_price = regressor.predict(X_test)

predicted_stock_price = sc.inverse_transform(predicted_stock_price)